In [80]:
from qiskit import Aer, QuantumCircuit
from qiskit.test.mock import FakeNairobi
from qiskit.circuit.library import EfficientSU2
from symmer.symplectic import random_PauliwordOp
import mthree
import numpy as np

In [81]:
backend = FakeNairobi() #Aer.get_backend('ibm_oslo')

In [82]:
n_qubits = 2

obs = random_PauliwordOp(2, 10)
print(obs)
qc = EfficientSU2(n_qubits).decompose()
qc.measure_all()
qc.draw()

0.806+0.818j ZY +
-0.861-0.009j XZ +
-0.070-0.750j XI +
-0.875+1.811j ZX +
-0.256+0.218j ZY +
0.254+0.228j II +
-0.707-1.159j ZI +
1.195-0.463j XI +
-0.184-1.203j YX +
-0.716-1.059j ZY


┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐     ┌──────────┐»
   q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├──■──┤ Ry(θ[8]) ├»
        ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤┌─┴─┐├──────────┤»
   q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├┤ X ├┤ Ry(θ[9]) ├»
        └──────────┘└──────────┘└───┘└──────────┘└──────────┘└───┘└──────────┘»
meas: 2/══════════════════════════════════════════════════════════════════════»
                                                                              »
«        ┌───────────┐     ┌───────────┐┌───────────┐ ░ ┌─┐   
«   q_0: ┤ Rz(θ[10]) ├──■──┤ Ry(θ[12]) ├┤ Rz(θ[14]) ├─░─┤M├───
«        ├───────────┤┌─┴─┐├───────────┤├───────────┤ ░ └╥┘┌─┐
«   q_1: ┤ Rz(θ[11]) ├┤ X ├┤ Ry(θ[13]) ├┤ Rz(θ[15]) ├─░──╫─┤M├
«        └───────────┘└───┘└───────────┘└───────────┘ ░  ║ └╥┘
«meas: 2/════════════════════════════════════════════════╩══╩═
«                                                        0  1

In [83]:
m3   = mthree.M3Mitigation(backend)
m3.cals_from_system(range(n_qubits))

In [84]:
m3

In [85]:
n_shots = 2**12
job = backend.run(qc.bind_parameters(np.random.random(qc.num_parameters)), shots=n_shots)
result = job.result()
raw_counts = result.get_counts()
raw_counts

{'01': 301, '11': 1101, '00': 1224, '10': 1470}

In [86]:
quasis = m3.apply_correction(raw_counts, range(n_qubits))

In [92]:
sum(quasis.values())

1.0

In [88]:
quasis.nearest_probability_distribution()

{'01': 0.059824121798929715,
 '11': 0.28500236850043403,
 '00': 0.2868225128164549,
 '10': 0.3683509968841815}

In [89]:
{op:coeff/n_shots for op,coeff in raw_counts.items()}

{'01': 0.073486328125,
 '11': 0.268798828125,
 '00': 0.298828125,
 '10': 0.35888671875}